In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import random
import time
from utils import *
from models import *
from attention import *

import torch
import torch.optim as optim
from torch.utils import data
from sklearn.model_selection import train_test_split
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
# Download the file
get_file()
path_to_file = 'spa-eng/spa.txt'

======== THE DATASET IS READY ========


In [4]:
# Try experimenting with the size of that dataset
num_examples = 30000#118960
input_tensor, target_tensor, inp_lang, targ_lang = load_dataset(path_to_file, num_examples)

# Calculate max_length of the target tensors
max_length_targ, max_length_inp = max_length(target_tensor), max_length(input_tensor)

The number of line in the dataset is 118964


In [5]:
# Creating training and validation sets using an 80-20 split
input_tensor_train, input_tensor_val, target_tensor_train, target_tensor_val = train_test_split(input_tensor, target_tensor, test_size=0.1)

# Show length
print(len(input_tensor_train), len(target_tensor_train), len(input_tensor_val), len(target_tensor_val))

27000 27000 3000 3000


In [6]:
#inp_lang.index_word

In [6]:
print ("Input Language; index to word mapping")
convert(inp_lang, input_tensor_train[0])
print ()
print ("Target Language; index to word mapping")
convert(targ_lang, target_tensor_train[0])

Input Language; index to word mapping
2 ----> <start>
31 ----> esto
53 ----> fue
294 ----> culpa
326 ----> mia
4 ----> .
3 ----> <end>

Target Language; index to word mapping
2 ----> <start>
17 ----> this
21 ----> was
18 ----> my
448 ----> fault
4 ----> .
3 ----> <end>


In [7]:
BATCH_SIZE = 64
steps_per_epoch = len(input_tensor_train)// BATCH_SIZE
embedding_dim = 256
units = 1024
vocab_inp_size = len(inp_lang.word_index)+1
vocab_tar_size = len(targ_lang.word_index)+1

In [8]:
# transform to torch tensor
tensor_x = torch.Tensor(input_tensor_train).long() 
tensor_y = torch.Tensor(target_tensor_train).long()
# create your datset
my_dataset = data.TensorDataset(tensor_x,tensor_y) 
# create your dataloader
my_dataloader = data.DataLoader(my_dataset,
                        batch_size=BATCH_SIZE,
                        shuffle=True,
                        drop_last=True,
                        num_workers=4)   

In [9]:
example_input_batch, example_target_batch = next(iter(my_dataloader))

In [10]:
example_input_batch.shape

torch.Size([64, 16])

In [11]:
encoder = Encoder(vocab_inp_size, embedding_dim, units, BATCH_SIZE)

# sample input
sample_hidden = encoder.initialize_hidden_state()
sample_output, sample_hidden = encoder(example_input_batch, sample_hidden)
print ('Encoder output shape: (batch size, sequence length, units) {}'.format(sample_output.shape))
print ('Encoder Hidden state shape: (batch size, units) {}'.format(sample_hidden.shape))

Encoder output shape: (batch size, sequence length, units) torch.Size([64, 16, 1024])
Encoder Hidden state shape: (batch size, units) torch.Size([1, 64, 1024])


In [12]:
attention_layer = BahdanauAttention(10, 1024)
attention_result, attention_weights = attention_layer(sample_hidden, sample_output)

print("Attention result shape: (batch size, units) {}".format(attention_result.shape))
print("Attention weights shape: (batch_size, sequence_length, 1) {}".format(attention_weights.shape))

Attention result shape: (batch size, units) torch.Size([64, 1024])
Attention weights shape: (batch_size, sequence_length, 1) torch.Size([64, 16, 1])


In [13]:
decoder = Decoder(vocab_tar_size, embedding_dim, units, BATCH_SIZE, 1024)

sample_decoder_output, _, _ = decoder(torch.randint(1, 20, (BATCH_SIZE, 1)),
                                      sample_hidden, sample_output)

print ('Decoder output shape: (batch_size, vocab size) {}'.format(sample_decoder_output.shape))

Decoder output shape: (batch_size, vocab size) torch.Size([64, 5109])


In [16]:
encoder = Encoder(vocab_inp_size, embedding_dim, units, BATCH_SIZE).to(device)
decoder = Decoder(vocab_tar_size, embedding_dim, units, BATCH_SIZE, 1024).to(device)

encoder_optimizer = optim.Adam(encoder.parameters())
decoder_optimizer = optim.Adam(decoder.parameters())

criterion = nn.NLLLoss()

In [17]:
# Train the model
EPOCHS = 2

for epoch in range(EPOCHS):
    start = time.time()

    total_loss = 0

    for (batch, (inp, targ)) in enumerate(my_dataloader):
        inp, targ = inp.to(device), targ.to(device)
        batch_loss = train_step(inp, targ, encoder, decoder,
                                encoder_optimizer, decoder_optimizer,
                                criterion, device, BATCH_SIZE, targ_lang)
        
        total_loss += batch_loss

        if batch % 100 == 0:
            print('Epoch {} Batch {} Loss {:.4f}'.format(epoch + 1, batch, batch_loss))
            
    # saving (checkpoint) the model every 2 epochs
    if (epoch + 1) % 2 == 0:
        
        pass

    print('Epoch {} Loss {:.4f}'.format(epoch + 1, total_loss / steps_per_epoch))
    print('Time taken for 1 epoch {} sec\n'.format(time.time() - start))


Epoch 1 Batch 0 Loss 7.7462
Epoch 1 Batch 100 Loss 3.0146
Epoch 1 Batch 200 Loss 1.7599
Epoch 1 Batch 300 Loss 1.4882
Epoch 1 Batch 400 Loss 1.1457
Epoch 1 Loss 2.0026
Time taken for 1 epoch 42.707030057907104 sec

Epoch 2 Batch 0 Loss 1.0166
Epoch 2 Batch 100 Loss 1.4357
Epoch 2 Batch 200 Loss 1.1084
Epoch 2 Batch 300 Loss 1.2062
Epoch 2 Batch 400 Loss 1.2135
Epoch 2 Loss 1.1259
Time taken for 1 epoch 42.873536825180054 sec



In [41]:
translate(u'trata de averiguarlo .', max_length_targ, max_length_inp, encoder,
          decoder, inp_lang, targ_lang, device, beam_width=10, alpha=0.1)

Input: <start> trata de averiguarlo . <end>
Predicted translation: <start> try to know . <end> 


In [42]:
translate(u'trata de averiguarlo .', max_length_targ, max_length_inp, encoder,
          decoder, inp_lang, targ_lang, device, beam_width=100, alpha=0.1)

Input: <start> trata de averiguarlo . <end>
Predicted translation: <start> try to know . <end> 


In [43]:
translate(u'trata de averiguarlo .', max_length_targ, max_length_inp, encoder,
          decoder, inp_lang, targ_lang, device, beam_search=False)

Input: <start> trata de averiguarlo . <end>
Predicted translation: i'll try to it . <end> 


In [44]:
translate(u'esta es mi vida .', max_length_targ, max_length_inp, encoder, decoder, inp_lang, targ_lang, device)

Input: <start> esta es mi vida . <end>
Predicted translation: <start> this is my life . <end> 


In [45]:
translate(u'esta es mi vida .', max_length_targ, max_length_inp, encoder,
          decoder, inp_lang, targ_lang, device, beam_width=10, alpha=0.2)

Input: <start> esta es mi vida . <end>
Predicted translation: <start> this is my life . <end> 


In [46]:
translate(u'hace mucho frio aqui .', max_length_targ, max_length_inp, encoder, decoder, inp_lang, targ_lang, device, beam_search=False)

Input: <start> hace mucho frio aqui . <end>
Predicted translation: it's really cold . <end> 


In [53]:

translate(u'hace mucho frio aqui .', max_length_targ, max_length_inp, encoder, decoder, inp_lang, targ_lang, device, beam_width=5, alpha=0.2)

Input: <start> hace mucho frio aqui . <end>
Predicted translation: <start> it really cold cold . <end> 


# FIN